In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.gold_serving.run_sst_result
USING DELTA
AS
SELECT
  r.time AS run_time,
  r.in_num,
  r.in_name,
  r.pk,

  s.number AS sst_number,
  s.in_num AS sst_in_num,
  s.in_name AS sst_in_name,
  s.sst_name,
  s.peak,
  s.eval_result,
  s.result AS sst_result,

  res.samp_id,
  res.as_id,
  res.Type,
  res.comp,
  res.unit,
  res.det_id,
  res.result AS assay_result,
  res.number_of_averaged_samples

FROM workspace.gold_serving.dimension_runs r
LEFT JOIN workspace.gold_serving.fact_sst s
  ON r.pk = s.fk
LEFT JOIN workspace.gold_serving.fact_results res
  ON r.pk = res.fk;

MERGE INTO workspace.gold_serving.run_sst_result AS target
USING (
  SELECT * FROM (
    SELECT
      r.time AS run_time,
      r.in_num,
      r.in_name,
      r.pk,

      s.number AS sst_number,
      s.in_num AS sst_in_num,
      s.in_name AS sst_in_name,
      s.sst_name,
      s.peak,
      s.eval_result,
      s.result AS sst_result,

      res.samp_id,
      res.as_id,
      res.Type,
      res.comp,
      res.unit,
      res.det_id,
      res.result AS assay_result,
      res.number_of_averaged_samples,

      ROW_NUMBER() OVER (
        PARTITION BY r.pk, s.number, res.samp_id
        ORDER BY r.time DESC
      ) AS rn
    FROM workspace.gold_serving.dimension_runs r
    LEFT JOIN workspace.gold_serving.fact_sst s ON r.pk = s.fk
    LEFT JOIN workspace.gold_se_
